In [1]:
import os

# Define the target directory
target_directory = r"C:\Users\pablosal\Desktop\gbb-ai-hls-factory-prior-auth"  # change your directory here

# Check if the directory exists
if os.path.exists(target_directory):
    # Change the current working directory
    os.chdir(target_directory)
    print(f"Directory changed to {os.getcwd()}")
else:
    print(f"Directory {target_directory} does not exist.")

Directory changed to C:\Users\pablosal\Desktop\gbb-ai-hls-factory-prior-auth


In [2]:
#!pip install --upgrade semantic_kernel

In [2]:
import asyncio

from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.connectors.ai.chat_completion_client_base import ChatCompletionClientBase
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.functions.kernel_arguments import KernelArguments
from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.azure_chat_prompt_execution_settings import (
    AzureChatPromptExecutionSettings,
)

In [3]:
from semantic_kernel import __version__

__version__

'1.17.0'

In [4]:
kernel = Kernel()

In [5]:
# Initialize the kernel
kernel = Kernel()

AZURE_OPENAI_KEY = os.getenv("AZURE_OPENAI_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AZURE_OPENAI_EMBEDDING_DEPLOYMENT = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT")
AZURE_OPENAI_CHAT_DEPLOYMENT_ID = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_ID")

service_id = "gpt-4o"
# Add Azure OpenAI chat completion
kernel.add_service(AzureChatCompletion(
    service_id=service_id,
    deployment_name=AZURE_OPENAI_CHAT_DEPLOYMENT_ID,
    api_key=AZURE_OPENAI_KEY,
    endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=AZURE_OPENAI_API_VERSION,
))

In [14]:
# # Define the parent directory and plugin name
# parent_directory = os.path.abspath("src\\agenticai\\plugins")
# plugin_name = "prompt"

# # Add the plugin to the kernel
# plugin = kernel.add_plugin(parent_directory=parent_directory, plugin_name=plugin_name)

# Define the parent directory and plugin name
# parent_directory = os.path.abspath("src\\agenticai\\plugins\\plugins_store")
# plugin_name = "retrieval"

# # Add the plugin to the kernel
# kernel.add_plugin(parent_directory=parent_directory, plugin_name=plugin_name)

# # Define the parent directory and plugin name
parent_directory = os.path.abspath("src\\agenticai\\plugins\\plugins_store\\")
plugin_name = "hello_world"

# Add the plugin to the kernel
plugin = kernel.add_plugin(parent_directory=parent_directory, plugin_name=plugin_name)

Failed to create function from directory: C:\Users\pablosal\Desktop\gbb-ai-hls-factory-prior-auth\src\agenticai\plugins\plugins_store\hello_world\plugins_store
Failed to create function from directory: C:\Users\pablosal\Desktop\gbb-ai-hls-factory-prior-auth\src\agenticai\plugins\plugins_store\hello_world\prompt


In [13]:
kernel.plugins

{'hello_world': KernelPlugin(name='hello_world', description=None, functions={'limerick': KernelFunctionFromPrompt(metadata=KernelFunctionMetadata(name='limerick', plugin_name='hello_world', description='Generate a funny limerick about a person', parameters=[KernelParameterMetadata(name='name', description='', default_value='Bob', type_='', is_required=True, type_object=None, schema_data={'type': 'object', 'description': '(default value: Bob)'}, include_in_function_choices=True), KernelParameterMetadata(name='input', description='', default_value='Dogs', type_='', is_required=True, type_object=None, schema_data={'type': 'object', 'description': '(default value: Dogs)'}, include_in_function_choices=True)], is_prompt=True, is_asynchronous=True, return_parameter=KernelParameterMetadata(name='return', description='The completion result', default_value=None, type_='FunctionResult', is_required=True, type_object=None, schema_data=None, include_in_function_choices=True), additional_properties

In [9]:
arguments = KernelArguments(topic="time travel to dinosaur age", length="super silly")
result = await kernel.invoke(plugin['GenerateStory'], arguments)
print(result)

In [13]:
chat_completion : AzureChatCompletion = kernel.get_service(type=ChatCompletionClientBase)

In [14]:
execution_settings = AzureChatPromptExecutionSettings()
execution_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

In [15]:
arguments = KernelArguments(search_text="the Prior authorization for Adalimumab for Crohn's Disease and Anemia due to blood loss. Related terms: Crohn's Disease, regional enteritis, inflammatory bowel disease, IBD, chronic enteritis, ileitis, anemia secondary to blood loss, iron deficiency anemia, hemorrhagic anemia. Medication: Adalimumab, Humira, anti-TNF therapy, biologic therapy, TNF inhibitor. Dosage: 160 mg initial dose (four 40 mg injections), 80 mg two weeks later (two 40 mg injections), 40 mg every other week. Duration: 6 months. Rationale: Initiation of biologic therapy due to severity of symptoms despite current treatment.")

In [16]:
# Create a history of the conversation
chat_history = ChatHistory()

# Add system message to the chat history to define the system's role
chat_history.add_system_message(
    "You are an AI-powered assistant that specializes in policy retrieval for prior authorization queries. "
    "Your task is to classify the query and select the best search strategy (keyword, semantic, or hybrid). "
    "Once the classification is complete, you will evaluate multiple potential policies, calculate their relevance, "
    "and return a JSON object with the following details:\n"
    "1. **List of Evaluated Policies**: Each policy includes its name, relevance score, and reasoning for its relevance.\n"
    "2. **Confidence Levels**: Each policy is given a confidence score based on its alignment with the search criteria.\n"
    "3. **Final Policy Selection**: The system will select the policy with the highest confidence score as the final policy.\n"
    "4. **JSON Response**: The final response will be a JSON object with the format:\n"
    "```json\n"
    "{\n"
    "  'evaluated_policies': [\n"
    "    {\n"
    "      'policy_name': '<name>',\n"
    "      'confidence_score': <float>,\n"
    "      'reasoning': '<explanation of why this policy was selected>'\n"
    "    },\n"
    "    ...\n"
    "  ],\n"
    "  'final_selected_policy': {\n"
    "    'policy_name': '<name>',\n"
    "    'confidence_score': <float>,\n"
    "    'reasoning': '<why this policy was selected as the best option>'\n"
    "  }\n"
    "}\n"
)

# User message containing a detailed query
chat_history.add_user_message(
    "I need the most relevant policy that matches my search criteria. "
    "Please classify the query and retrieve the best-matching policy using the most effective search strategy. "
    "Here is the search query: Prior authorization for Adalimumab for Crohn's Disease and Anemia due to blood loss. "
    "Related terms: Crohn's Disease, regional enteritis, inflammatory bowel disease (IBD), chronic enteritis, ileitis, "
    "anemia secondary to blood loss, iron deficiency anemia, hemorrhagic anemia. "
    "Medication: Adalimumab, Humira, anti-TNF therapy, biologic therapy, TNF inhibitor. "
    "Dosage: 160 mg initial dose (four 40 mg injections), 80 mg two weeks later (two 40 mg injections), 40 mg every other week. "
    "Duration: 6 months. "
    "Rationale: Initiation of biologic therapy due to the severity of symptoms despite current treatment. "
    "Please evaluate this query using all available retrieval methods to determine the most effective classification and strategy. "
    "Once you have the classification, use it to select one of the following retrieval strategies: **keyword**, **semantic**, or **hybrid** search.\n\n"
    "Evaluate all the retrieved policies and calculate their confidence scores. Return a detailed JSON object as follows:\n"
    "1. **List of Evaluated Policies**: Each policy includes its name, relevance score, and reasoning for its relevance.\n"
    "2. **Confidence Levels**: Each policy is assigned a confidence score from 0 to 1.\n"
    "3. **Final Policy Selection**: The system selects the policy with the highest confidence score.\n"
    "4. **Return the JSON Object**: The response should be a JSON object with the following structure:\n"
    "```json\n"
    "{\n"
    "  'evaluated_policies': [\n"
    "    {\n"
    "      'policy_name': '<name>',\n"
    "      'confidence_score': <float>,\n"
    "      'reasoning': '<explanation of why this policy was selected>'\n"
    "    },\n"
    "    ...\n"
    "  ],\n"
    "  'final_selected_policy': {\n"
    "    'policy_name': '<name>',\n"
    "    'confidence_score': <float>,\n"
    "    'reasoning': '<why this policy was selected as the best option>'\n"
    "  }\n"
    "}\n"
)

import logging

# Set the logging level for  semantic_kernel.kernel to DEBUG.
logging.basicConfig(
    format="[%(asctime)s - %(name)s:%(lineno)d - %(levelname)s] %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

from semantic_kernel.utils.logging import setup_logging
setup_logging()
logging.getLogger("kernel").setLevel(logging.INFO)


# Get the response from the AI
result = (await chat_completion.get_chat_message_contents(
    chat_history=chat_history,
    settings=execution_settings,
    kernel=kernel,
    arguments=KernelArguments(),
))[0]

2024-12-16 09:15:16,648 - AIQueryClassificationPlugin - MainProcess - INFO     Classifying query: Prior authorization for Adalimumab for Crohn's Disease and Anemia due to blood loss. Related terms: Crohn's Disease, regional enteritis, inflammatory bowel disease (IBD), chronic enteritis, ileitis, anemia secondary to blood loss, iron deficiency anemia, hemorrhagic anemia. Medication: Adalimumab, Humira, anti-TNF therapy, biologic therapy, TNF inhibitor. Dosage: 160 mg initial dose (four 40 mg injections), 80 mg two weeks later (two 40 mg injections), 40 mg every other week. Duration: 6 months. Rationale: Initiation of biologic therapy due to the severity of symptoms despite current treatment. (aisearchclassification.py:classify_query:109)
[2024-12-16 09:15:16 - AIQueryClassificationPlugin:109 - INFO] Classifying query: Prior authorization for Adalimumab for Crohn's Disease and Anemia due to blood loss. Related terms: Crohn's Disease, regional enteritis, inflammatory bowel disease (IBD), 

In [15]:
result

ChatMessageContent(inner_content=ChatCompletion(id='chatcmpl-Aendd842ly7Q2blnqGeGf3SCSlufZ', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="The most relevant policy matching your criteria is from the **Cigna National Formulary Coverage - Policy: Inflammatory Conditions - Adalimumab Products Prior Authorization**. Here are the key excerpts from the document that align with your query:\n\n1. **Requirement for Trials Before Starting Adalimumab**:\n   - Patients must have a history of trying steroids or another conventional systemic agent unless they have previously tried at least one biologic other than the requested medication. \n   - Mesalamine does not count as a systemic agent for Crohn's disease.\n   - Patients may have enterocutaneous (perianal or abdominal) or rectovaginal fistulas.\n\n2. **Clinical Response Requirements**:\n   - Beneficial clinical response is defined as improvement in at least one of the following parameters: 

In [15]:
print(result)

Based on the hybrid search results, the system evaluated the relevant policies and calculated their confidence scores. Here is the detailed response:

```json
{
  "evaluated_policies": [
    {
      "policy_name": "001.pdf - Page 5",
      "confidence_score": 0.85,
      "reasoning": "The policy specifically mentions requirements for Crohn's Disease, use of corticosteroids, and alternative systemic therapies. The duration for initial therapy is also specified as 6 months, which aligns well with the query."
    },
    {
      "policy_name": "001.pdf - Page 6",
      "confidence_score": 0.80,
      "reasoning": "The policy discusses approval criteria for patients currently receiving Adalimumab, including Crohn's Disease and related requirements. This includes ongoing therapy for at least 6 months."
    },
    {
      "policy_name": "001.pdf - Page 10",
      "confidence_score": 0.75,
      "reasoning": "This policy section mentions therapy conditions for patients with Crohn's Disease, in

## Custom Agents

In [19]:
import semantic_kernel as sk
from semantic_kernel.functions import KernelArguments
from semantic_kernel.functions import kernel_function
import asyncio
from typing import List, Dict, Any

# Step 1: Define Agent Configuration and Agent Classes
class AgentConfig:
    """Configuration for an Agent."""
    def __init__(self, name: str, role: str, skills: List[str]):
        self.name = name
        self.role = role
        self.skills = skills

class Agent:
    """Class for an Agent that can interact with the Semantic Kernel."""
    def __init__(self, config: AgentConfig, kernel: sk.Kernel):
        self.name = config.name
        self.role = config.role
        self.skills = config.skills
        self.kernel = kernel
        self.context = {}  # Dictionary to store context variables

    def register_skills(self):
        """Load all skills for the agent."""
        for skill_name in self.skills:
            self.kernel.import_skill(skill_name)

In [4]:
class Skills:
    @kernel_function
    def search_skill(self, query: str, context: Dict[str, Any]) -> Dict[str, Any]:
        """Search Skill: Queries Azure AI Search to retrieve relevant content."""
        # Placeholder logic for search
        print(f"Executing search skill with query: {query}")
        return {"content": f"Results for {query}"}

    @kernel_function
    def validation_skill(self, document_content: str, reference_content: str) -> bool:
        """Validation Skill: Validates retrieved content against a reference document."""
        print("Validating document content against reference content.")
        return document_content in reference_content  # Simple validation logic

    @kernel_function
    def query_rewrite_skill(self, original_query: str, feedback: str) -> str:
        """Query Rewrite Skill: Modifies the query based on feedback from validation."""
        print("Rewriting query based on feedback.")
        return f"{original_query} {feedback}"  # Simple concatenation for demonstration purposes


ModuleNotFoundError: No module named 'semantic_kernel.orchestration'

In [ ]:
# Agent 1: Document Retrieval Agent
retrieval_agent_config = AgentConfig(
    name="DocumentRetrievalAgent",
    role="Retrieve relevant documents from Azure AI Search",
    skills=["search_skill"]
)
retrieval_agent = Agent(config=retrieval_agent_config, kernel=kernel)
retrieval_agent.register_skills()

# Agent 2: Validation Agent
validation_agent_config = AgentConfig(
    name="DocumentValidationAgent",
    role="Check document accuracy against original content",
    skills=["validation_skill"]
)
validation_agent = Agent(config=validation_agent_config, kernel=kernel)
validation_agent.register_skills()

# Agent 3: Query Rewrite Agent
query_rewrite_agent_config = AgentConfig(
    name="QueryRewriteAgent",
    role="Rewrite search query based on feedback",
    skills=["query_rewrite_skill"]
)
query_rewrite_agent = Agent(config=query_rewrite_agent_config, kernel=kernel)
query_rewrite_agent.register_skills()

In [9]:
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.agents import AgentGroupChat, ChatCompletionAgent
from semantic_kernel.agents.strategies import (
    KernelFunctionSelectionStrategy,
    KernelFunctionTerminationStrategy,
)
from semantic_kernel.connectors.ai.open_ai.services.azure_chat_completion import AzureChatCompletion
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.functions.kernel_function_from_prompt import KernelFunctionFromPrompt
from semantic_kernel.kernel import Kernel


In [18]:
ChatCompletionAgent?

Init signature:
ChatCompletionAgent(
    service_id: str | None = None,
    kernel: 'Kernel | None' = None,
    name: str | None = None,
    id: str | None = None,
    description: str | None = None,
    instructions: str | None = None,
    execution_settings: semantic_kernel.connectors.ai.prompt_execution_settings.PromptExecutionSettings | None = None,
) -> None
Docstring:     
A KernelAgent specialization based on ChatCompletionClientBase.

    Note: enable `function_choice_behavior` on the PromptExecutionSettings to enable function
    choice behavior which allows the kernel to utilize plugins and functions registered in
    the kernel.
    

Note: This class is experimental and may change in the future.
Init docstring:
Initialize a new instance of ChatCompletionAgent.

Args:
    service_id: The service id for the chat completion service. (optional) If not provided,
        the default service name `default` will be used.
    kernel: The kernel instance. (optional)
    name: The nam

In [10]:
from semantic_kernel.agents import ChatCompletionAgent

# NOTE: This is all that is required to enable logging
logging.basicConfig(level=logging.DEBUG)

FORMULATOR_NAME = "Formulator"
FORMULATOR_INSTRUCTIONS = """
You are responsible for formulating the query based on the given clinical metadata.
Your job is to analyze the metadata, identify key details, and create an optimized search query.
Output only the query string with no additional text.
"""

# Agent Instructions
RETRIEVER_NAME = "Retriever"
RETRIEVER_INSTRUCTIONS = """
You are responsible for retrieving content from Azure AI Search.
Use the provided query to search the database and return the top document results.
Please make sure to determine the retrieval strategy based on the query. Classify the query first and then retrieve the information.
If there are no results, state 'No relevant content found.'
"""

# Agent Instructions
EVALUATOR_NAME = "Evaluator"
EVALUATOR_INSTRUCTIONS = """
You are responsible for evaluating if the retrieved content is accurate and matches the original clinical requirements.
If the content is acceptable, respond 'Approved.'
If the content is not accurate or does not solve the issue, suggest improvements without examples.
"""

def _create_kernel_with_chat_completion(service_id: str) -> Kernel:
    kernel = Kernel()
    AZURE_OPENAI_KEY = os.getenv("AZURE_OPENAI_KEY")
    AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
    AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
    AZURE_OPENAI_EMBEDDING_DEPLOYMENT = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT")
    AZURE_OPENAI_CHAT_DEPLOYMENT_ID = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_ID")

    service_id=service_id
    # Add Azure OpenAI chat completion
    kernel.add_service(AzureChatCompletion(
        service_id=service_id,
        deployment_name=AZURE_OPENAI_CHAT_DEPLOYMENT_ID,
        api_key=AZURE_OPENAI_KEY,
        endpoint=AZURE_OPENAI_ENDPOINT,
        api_version=AZURE_OPENAI_API_VERSION,
    ))
    return kernel


agent_formulator = ChatCompletionAgent(
    service_id=FORMULATOR_NAME,
    kernel=_create_kernel_with_chat_completion(FORMULATOR_NAME),
    name=FORMULATOR_NAME,
    instructions=FORMULATOR_INSTRUCTIONS,
)

kernel_retriever = Kernel()
kernel_retriever.add_service(AzureChatCompletion(
    service_id=RETRIEVER_NAME,
    deployment_name=AZURE_OPENAI_CHAT_DEPLOYMENT_ID,
    api_key=AZURE_OPENAI_KEY,
    endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=AZURE_OPENAI_API_VERSION,
))
parent_directory = os.path.abspath("src\\agenticai\\plugins\\plugins_store")

plugin_name = "retrieval"

# Add the plugin to the kernel
kernel.add_plugin(parent_directory=parent_directory, plugin_name=plugin_name)

agent_retriever = ChatCompletionAgent(
    service_id=RETRIEVER_NAME,
    kernel=_create_kernel_with_chat_completion(RETRIEVER_NAME),
    name=RETRIEVER_NAME,
    instructions=RETRIEVER_INSTRUCTIONS,
)

agent_evaluator = ChatCompletionAgent(
    service_id=EVALUATOR_NAME,
    kernel=_create_kernel_with_chat_completion(EVALUATOR_NAME),
    name=EVALUATOR_NAME,
    instructions=EVALUATOR_INSTRUCTIONS,
)


termination_function = KernelFunctionFromPrompt(
    function_name="termination",
    prompt="""
    Determine if the content evaluation has been approved. If so, respond 'yes.'
    If not, state 'no.'

    History:
    {{$history}}
    """,
)

DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='C:\\Users\\pablosal\\AppData\\Local\\anaconda3\\envs\\pa-ai-env\\Library\\ssl\\cacert.pem'
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='C:\\Users\\pablosal\\AppData\\Local\\anaconda3\\envs\\pa-ai-env\\Library\\ssl\\cacert.pem'
DEBUG:semantic_kernel.functions.kernel_plugin:Found object: C:\Users\pablosal\Desktop\gbb-ai-hls-factory-prior-auth\src\agenticai\plugins\plugins_store\retrieval\aisearch.py
DEBUG:semantic_kernel.functions.kernel_function_decorator:Parsing decorator for function: keyword_search
DEBUG:semantic_kernel.functions.kernel_function_decorator:Parsing param: search_text
DEBUG:semantic_kernel.functions.kernel_function_decorator:Parsing annotation: <class 'str'>
DEBUG:semantic_kernel.functions.kernel_function_decorator:Parsing param: top
DEBUG:semantic_kernel.functions.kernel_functi

In [12]:
selection_function = KernelFunctionFromPrompt(
    function_name="selection",
    prompt=f"""
    Determine which participant takes the next turn in a conversation based on the the most recent participant.
    State only the name of the participant to take the next turn.
    No participant should take more than one turn in a row.

    Choose only from these participants:
        - {FORMULATOR_NAME}
        - {RETRIEVER_NAME}
        - {EVALUATOR_NAME}

    Always follow these rules when selecting the next participant:
    - {FORMULATOR_NAME} starts first and formulates the query.
    - After {FORMULATOR_NAME}, it is {RETRIEVER_NAME}'s turn to retrieve the document.
    - After {RETRIEVER_NAME}, it is {EVALUATOR_NAME}'s turn to evaluate the content.
    - After {EVALUATOR_NAME}, the workflow may terminate if approved.

    History:
    {{{{$history}}}}
    """,
)

chat = AgentGroupChat(
    agents=[agent_formulator, agent_retriever, agent_evaluator],
    termination_strategy=KernelFunctionTerminationStrategy(
        agents=[agent_evaluator],
        function=termination_function,
        kernel=_create_kernel_with_chat_completion("termination"),
        result_parser=lambda result: str(result.value[0]).lower() == "yes",
        history_variable_name="history",
        maximum_iterations=10,
    ),
    selection_strategy=KernelFunctionSelectionStrategy(
        function=selection_function,
        kernel=_create_kernel_with_chat_completion("selection"),
        result_parser=lambda result: str(result.value[0]) if result.value is not None else FORMULATOR_NAME,
        agent_variable_name="agents",
        history_variable_name="history",
    ),
)

clinical_metadata = """
Patient with persistent lower back pain.
Searching for policies related to chronic pain management and suitable treatments.
"""

clinical_metadata = "I need the most relevant policy that matches my search criteria. "
"Please classify the query and retrieve the best-matching policy using the most effective search strategy. "
"Here is the clinical_metadata: Prior authorization for Adalimumab for Crohn's Disease and Anemia due to blood loss. "
"Related terms: Crohn's Disease, regional enteritis, inflammatory bowel disease (IBD), chronic enteritis, ileitis, "
"anemia secondary to blood loss, iron deficiency anemia, hemorrhagic anemia. "
"Medication: Adalimumab, Humira, anti-TNF therapy, biologic therapy, TNF inhibitor. "
"Dosage: 160 mg initial dose (four 40 mg injections), 80 mg two weeks later (two 40 mg injections), 40 mg every other week. "
"Duration: 6 months. "
"Rationale: Initiation of biologic therapy due to the severity of symptoms despite current treatment. "
"Please evaluate this query using all available retrieval methods to determine the most effective classification and strategy. "
"Once you have the classification, use it to select one of the following retrieval strategies: **keyword**, **semantic**, or **hybrid** search.\n\n"
"Evaluate all the retrieved policies and calculate their confidence scores. Return a detailed JSON object as follows:\n"
"1. **List of Evaluated Policies**: Each policy includes its name, relevance score, and reasoning for its relevance.\n"
"2. **Confidence Levels**: Each policy is assigned a confidence score from 0 to 1.\n"
"3. **Final Policy Selection**: The system selects the policy with the highest confidence score.\n"
"4. **Return the JSON Object**: The response should be a JSON object with the following structure:\n"
"```json\n"
"{\n"
"  'evaluated_policies': [\n"
"    {\n"
"      'policy_name': '<name>',\n"
"      'confidence_score': <float>,\n"
"      'reasoning': '<explanation of why this policy was selected>'\n"
"    },\n"
"    ...\n"
"  ],\n"
"  'final_selected_policy': {\n"
"    'policy_name': '<name>',\n"
"    'confidence_score': <float>,\n"
"    'reasoning': '<why this policy was selected as the best option>'\n"
"  }\n"
"}\n"

# Start the chat with Formulator using user input
await chat.add_chat_message(ChatMessageContent(role=AuthorRole.USER, content=clinical_metadata))
print(f"# {AuthorRole.USER}: '{clinical_metadata}'")

# Iterate through the agent group chat workflow
async for content in chat.invoke():
    print(f"# {content.role} - {content.name or '*'}: '{content.content}'")

print(f"# Workflow Complete: {chat.is_complete}")

DEBUG:semantic_kernel.prompt_template.kernel_prompt_template:Extracting blocks from template: 
    Determine which participant takes the next turn in a conversation based on the the most recent participant.
    State only the name of the participant to take the next turn.
    No participant should take more than one turn in a row.

    Choose only from these participants:
        - Formulator
        - Retriever
        - Evaluator

    Always follow these rules when selecting the next participant:
    - Formulator starts first and formulates the query.
    - After Formulator, it is Retriever's turn to retrieve the document.
    - After Retriever, it is Evaluator's turn to evaluate the content.
    - After Evaluator, the workflow may terminate if approved.

    History:
    {{$history}}
    
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='C:\\Users\\pablosal\\AppData\\Local\\anaconda3\\envs\\pa-ai-env\\Library\\ssl

# user: 'I need the most relevant policy that matches my search criteria. '


DEBUG:httpcore.connection:start_tls.complete return_value=<httpcore._backends.anyio.AnyIOStream object at 0x0000021E446A1300>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Content-Length', b'333'), (b'Content-Type', b'application/json'), (b'apim-request-id', b'0797afe8-722b-402d-99b1-e9e51ef0f776'), (b'Strict-Transport-Security', b'max-age=31536000; includeSubDomains; preload'), (b'x-ratelimit-remaining-requests', b'149'), (b'x-accel-buffering', b'no'), (b'x-ms-rai-invoked', b'true'), (b'x-request-id', b'51c23019-f962-47e5-b0a3-741b6034aa3a'), (b'x-content-type-options', b'nosniff'), (b'azureml-model-

# assistant - Formulator: 'site:gov OR site:edu policy relevant criteria'


DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Content-Length', b'332'), (b'Content-Type', b'application/json'), (b'apim-request-id', b'c9bb985f-c68c-4b06-aaa5-580e50845bb7'), (b'Strict-Transport-Security', b'max-age=31536000; includeSubDomains; preload'), (b'x-ratelimit-remaining-requests', b'147'), (b'x-accel-buffering', b'no'), (b'x-ms-rai-invoked', b'true'), (b'x-request-id', b'1729d506-d95f-4075-89b5-64de44c393fe'), (b'x-content-type-options', b'nosniff'), (b'azureml-model-session', b'd097-20241115065003'), (b'x-ms-region', b'East US'), (b'x-envoy-upstream-service-time', b'157'), (b'x-ms-client-request-id', b'c9bb985f-c68c-4b06-aaa5-580e50845bb7'), (b'x-ratelimit-remaining-tokens', b'147583'), (b'Date', b'Mon, 16 Dec 2024 04:14:19 GMT')])
INFO:httpx:HTTP Request: POST https://aoai-ai-factory-eus-dev.openai.azure.com/openai/deployments/gpt-4o-standard/chat/completions?api-version=2023-05-15 "HTTP/1.1 200 OK"
DEBUG:httpcore.http11:

# assistant - Retriever: 'Could you please provide the specific criteria or the topic that you are interested in? This will help me in retrieving the most relevant policy for you.'


DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Content-Length', b'332'), (b'Content-Type', b'application/json'), (b'apim-request-id', b'59ca286d-8da3-40da-b309-183dd5b11d4b'), (b'Strict-Transport-Security', b'max-age=31536000; includeSubDomains; preload'), (b'x-ratelimit-remaining-requests', b'145'), (b'x-accel-buffering', b'no'), (b'x-ms-rai-invoked', b'true'), (b'x-request-id', b'446f01f7-be0f-4a16-8765-f3fea671f9d5'), (b'x-content-type-options', b'nosniff'), (b'azureml-model-session', b'd100-20241115101253'), (b'x-ms-region', b'East US'), (b'x-envoy-upstream-service-time', b'149'), (b'x-ms-client-request-id', b'59ca286d-8da3-40da-b309-183dd5b11d4b'), (b'x-ratelimit-remaining-tokens', b'145912'), (b'Date', b'Mon, 16 Dec 2024 04:14:20 GMT')])
INFO:httpx:HTTP Request: POST https://aoai-ai-factory-eus-dev.openai.azure.com/openai/deployments/gpt-4o-standard/chat/completions?api-version=2023-05-15 "HTTP/1.1 200 OK"
DEBUG:httpcore.http11:

# assistant - Evaluator: 'Approved.'
# Workflow Complete: True


In [ ]:
# Step 5: Multi-Agent Orchestrator

class MultiAgentOrchestrator:
    """Main Orchestrator to manage all agents and task execution."""
    def __init__(self, kernel: sk.Kernel):
        self.kernel = kernel
        self.agents = {}
        self.planner = sk.Planning.SequentialPlanner(kernel)  # Orchestrates step-by-step task execution
    
    def register_agent(self, agent: Agent):
        """Add agent to the system."""
        self.agents[agent.name] = agent

    async def execute_workflow(self, objective: str):
        """Execute workflow to achieve an objective."""
        print("Creating a plan for objective:", objective)
        plan = await self.planner.create_plan(objective)
        print("Plan created. Executing steps.")
        results = await self.execute_plan(plan)
        return results

    async def execute_plan(self, plan):
        """Execute the step-by-step plan."""
        for step in plan.steps:
            agent_name = step.target_agent
            if agent_name in self.agents:
                agent = self.agents[agent_name]
                context = agent.context
                context.update(step.variables)
                result = await agent.kernel.execute(step.action, context)
                print(f"Result from {agent_name}: {result}")
        return result


# Step 6: Register Agents with the Orchestrator

orchestrator = MultiAgentOrchestrator(kernel=kernel)

# Register all agents
orchestrator.register_agent(retrieval_agent)
orchestrator.register_agent(validation_agent)
orchestrator.register_agent(query_rewrite_agent)


# Step 7: Execute the Workflow

async def main():
    objective = "Retrieve prior authorization policy for diabetes treatment and validate against the original document"
    print("Starting workflow for objective:", objective)
    final_result = await orchestrator.execute_workflow(objective)
    print(f"Final Result: {final_result}")

if __name__ == "__main__":
    asyncio.run(main())